### &#x1F3C5; FIRST TRAINING WITH XGBRANKER

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

ImportError: initialization failed

In [4]:
df_train = pd.read_csv('../data/train_10k.csv').drop(columns=['Unnamed: 0'])
df_val = pd.read_csv('../data/val_10k.csv').drop(columns=['Unnamed: 0'])

In [6]:
import xgboost as xgb

In [7]:
from sklearn.metrics import make_scorer

In [8]:
# metric
from bisect import bisect

def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max

#def kendall_tau_for_xgb(y_pred, dtrain):
    #labels = dtrain.get_label()
    #return 'kendall_tau', kendall_tau(labels, y_pred)

kendall_tau_metric = make_scorer(kendall_tau)

In [9]:
X_train = df_train.drop(columns='rank')
X_val = df_val.drop(columns='rank')

y_train = df_train['rank']
y_val = df_val['rank']

# Groupes pour l'entraînement
groups_train = df_train.groupby('id').size().to_numpy()
groups_val = df_val.groupby('id').size().to_numpy()

model = xgb.XGBRanker(objective='rank:pairwise', min_child_weight=10, subsample=0.5, tree_method='hist')

# Entraîner le modèle
X_train.drop(columns=['id', 'cell_id'], inplace=True)
X_val.drop(columns=['id', 'cell_id'], inplace=True)
model.fit(X_train, y_train, group=groups_train)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=10, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, num_parallel_tree=None, objective='rank:pairwise', ...)

#### &#x1F3C7; L'entrainement est très rapide car on fait du data leakage intentionnellement sur 66% de la donnée, on donne le rang des cellules de code. 

#### 66% code cells, 34% markdown

In [10]:
y_pred = model.predict(X_val)

In [11]:
predict = pd.DataFrame({"cell_id" : df_val['cell_id'],
                           "pred" : y_pred, 'id' : df_val['id']})
predict.set_index('id', inplace=True)

In [12]:
# Sort (using the predicted rank) and then group
predict = predict.sort_values(by = ['id', 'pred'], ascending = [False, True]).groupby('id')['cell_id'].apply(list)
# Create the same but for actual data
actual = df_val.sort_values(by = ['id', 'rank'], ascending = [False, True]).groupby('id')['cell_id'].apply(list)
baseline = kendall_tau(actual, predict)

In [2]:
y_pred

NameError: name 'y_pred' is not defined

In [13]:
baseline

0.5726563188617604

#### &#x1F622; Le score est très nul 

In [18]:
#from sklearn.model_selection import RandomizedSearchCV, GroupKFold
#from scipy import stats

In [19]:
#param_dist = {'n_estimators': stats.randint(40, 1000),
#              'learning_rate': stats.uniform(0.01, 0.59),
#              'subsample': stats.uniform(0.3, 0.6),
#              'max_depth': [3, 4, 5, 6, 7, 8, 9],
#              'min_child_weight': [5, 10, 20]
#              }

#kfold = GroupKFold(n_splits=5)

#clf = RandomizedSearchCV(model,
#                         param_distributions=param_dist,
#                         cv=kfold,
#                         n_iter=5,
#                         scoring=kendall_tau_metric,
#                         error_score=0,
#                         verbose=3,
#                         n_jobs=-1)

In [23]:
#clf.fit(X_train, y_train, group=groups_train, groups=groups_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


ValueError: Found input variables with inconsistent numbers of samples: [374073, 374073, 8092]